# CIFAR-100 Prediction - Evaluation

This notebook is used to check the results and trained models and to test their quality.

## Data Loading

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras

In [ ]:
classes = tf.io.read_file("../data/raw/classes.txt")
classes = tf.strings.split(classes, "\n")

In [ ]:
train_dataset = (tf.data.Dataset.load("../data/splitted/train")
    .batch(32)
    .prefetch(tf.data.AUTOTUNE))

test_dataset = (tf.data.Dataset.load("../data/raw/test")
    .batch(32)
    .prefetch(tf.data.AUTOTUNE))

## Model Evaluation

### Convolutional Neural Network

In [ ]:
cnn_model = keras.saving.load_model("../models/cnn_model.keras")
cnn_history = np.load("../models/cnn_history.npy", allow_pickle=True).item()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(1, 1, 1)

epochs = range(1, len(cnn_history["accuracy"]) + 1)

ax.plot(epochs, cnn_history["accuracy"], label="accuracy")
ax.plot(epochs, cnn_history["val_accuracy"], label="val_accuracy")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy")
ax.set_xticks(epochs)
ax.legend()

plt.show()

In [ ]:
cnn_train_accuracy = cnn_model.evaluate(train_dataset, verbose=2)
cnn_test_accuracy = cnn_model.evaluate(test_dataset, verbose=2)

print(f"Train accuracy: {cnn_train_accuracy[1]}")
print(f"Test accuracy: {cnn_test_accuracy[1]}")

## Model Prediction

In [ ]:
final_model = cnn_model

In [ ]:
fig = plt.figure(figsize=(10, 10))
batch_images, batch_classes = next(iter(test_dataset.take(1)))
predictions = final_model.predict(batch_images)

for index in range(25):
    ax = plt.subplot(5, 5, index + 1)

    predicted_class = np.argmax(predictions[index])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(batch_images[index])
    ax.set_xlabel(f"{classes[predicted_class]} ({round(float(predictions[index][predicted_class]), 3)})")

plt.show()